### Effet du support

Ce graphique interactif illustre de façon concrète l’effet du **changement d’échelle (support)** sur la **distribution des teneurs**.

Au départ, deux gisements sont simulés avec une **distribution statistique identique** à l’échelle **ponctuelle (1 m × 1 m)**.  
Mais que se passe-t-il lorsqu’on augmente la taille des blocs ?

🔍 Grâce au **widget interactif ci-dessous**, vous pouvez ajuster la taille du **bloc de support** et observer en temps réel les conséquences :  
Par exemple, un bloc de **10 m × 10 m** est simplement la **moyenne des 100 cellules** de **1 m × 1 m** qu’il contient.

🗺️ Les **cartes**, 📊 les **histogrammes**, et 📈 les **fonctions de répartition cumulées** sont mis à jour automatiquement pour refléter les nouvelles propriétés statistiques du gisement après agrégation.

💡 C’est une façon simple mais puissante de visualiser l’**effet de support spatial** — un concept clé en estimation minière.

🎯 **Prenez quelques minutes pour explorer... et préparez vos questions pour la discussion en classe !**


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import VBox, HBox, IntText, Button, Output
from numpy.fft import fft2, ifft2, fftshift

# --- Fonctions (inchangées) ---
def spherical_covariance_fft(size, range_, sill=1.0):
    extended_size = 2 * size
    x = np.arange(-extended_size//2, extended_size//2)
    X, Y = np.meshgrid(x, x)
    h = np.sqrt(X**2 + Y**2)
    h = np.minimum(h, range_ * size)
    cov = sill * (1 - 1.5 * (h / (range_ * size)) + 0.5 * (h / (range_ * size))**3)
    cov[h > (range_ * size)] = 0
    return fftshift(cov)

def fftma_simulation(size, range_, sill=1.0, seed=0):
    np.random.seed(seed)
    extended_size = 2 * size
    cov_model = spherical_covariance_fft(size, range_, sill)
    cov_fft = fft2(cov_model)
    white_noise = np.random.normal(size=(extended_size, extended_size))
    white_fft = fft2(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    z_ext = np.real(ifft2(z_fft))
    start = extended_size // 4
    end = start + size
    return z_ext[start:end, start:end]

def gaussian_to_lognormal(field):
    return np.exp(field)

def match_histogram(reference, target):
    flat_ref = np.sort(reference.ravel())
    sorted_idx = np.argsort(target.ravel())
    result = np.zeros_like(target.ravel())
    result[sorted_idx] = flat_ref
    return result.reshape(target.shape)

def aggregate(field, block_size):
    s = field.shape[0]
    reduced_size = s // block_size
    aggregated = np.zeros((reduced_size, reduced_size))
    for i in range(reduced_size):
        for j in range(reduced_size):
            block = field[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            aggregated[i, j] = np.mean(block)
    return aggregated

# --- Préparation des données initiales ---
size = 500
range_short = 10/size
range_long = 250/size

gauss_short = fftma_simulation(size=size, range_=range_short, seed=42)
gauss_long = fftma_simulation(size=size, range_=range_long, seed=24)

lognorm_short = gaussian_to_lognormal(gauss_short)
lognorm_long = gaussian_to_lognormal(gauss_long)
lognorm_long = match_histogram(lognorm_short, lognorm_long)

# --- Affichage avec bouton ---
def plot_fields(support):
    fig, axes = plt.subplots(2, 2, figsize=(10, 7))

    agg_short = aggregate(lognorm_short, support)
    agg_long = aggregate(lognorm_long, support)

    vmin, vmax = 0, 10
    bins = np.linspace(0, 10, 11)
    mean = np.mean(agg_short)

    im0 = axes[0, 0].imshow(agg_short, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[0, 0].set_title(f'Portée courte – Support {support}×{support}', fontsize=12)
    axes[0, 0].axis('off')
    fig.colorbar(im0, ax=axes[0, 0], fraction=0.046, pad=0.04)

    im1 = axes[0, 1].imshow(agg_long, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[0, 1].set_title(f'Portée longue – Support {support}×{support}', fontsize=12)
    axes[0, 1].axis('off')
    fig.colorbar(im1, ax=axes[0, 1], fraction=0.046, pad=0.04)

    axes[1, 0].hist(agg_short.ravel(), bins=bins, alpha=0.6, color='steelblue', label='Portée courte', edgecolor='black')
    axes[1, 0].hist(agg_long.ravel(), bins=bins, alpha=0.6, color='darkorange', label='Portée longue', edgecolor='black')
    axes[1, 0].axvline(mean, color='black', linestyle='--', linewidth=2, label=f'Moyenne = {mean:.2f}')
    axes[1, 0].set_title('Histogrammes superposés')
    axes[1, 0].set_xlim([0, 10])
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    for data, color, label in zip([agg_short.ravel(), agg_long.ravel()],
                                   ['steelblue', 'darkorange'],
                                   ['Portée courte', 'Portée longue']):
        sorted_data = np.sort(data)
        ecdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
        axes[1, 1].plot(sorted_data, ecdf, color=color, lw=2, label=label)

    axes[1, 1].axvline(mean, color='black', linestyle='--', linewidth=2, label=f'Moyenne = {mean:.2f}')
    axes[1, 1].set_title('Fonctions de répartition cumulées')
    axes[1, 1].set_xlim([0, 10])
    axes[1, 1].set_ylim([0, 1])
    axes[1, 1].grid(True)
    axes[1, 1].legend()

    plt.tight_layout()
    plt.show()

# --- Widgets
support_input = IntText(value=1, description="Support (m):", min=1)
calc_button = Button(description="Calculer", button_style="success")
output = Output()

def on_calculate_clicked(b):
    with output:
        output.clear_output()
        support = support_input.value
        plot_fields(support)

calc_button.on_click(on_calculate_clicked)

# --- Affichage combiné
VBox([HBox([support_input, calc_button]), output])